[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1jL3JmZ-oXWZanlS8KI3i5rdVrwAaSdSe?usp=sharing)

# CrewAI Tutorial Analysis
This notebook demonstrates how to use CrewAI to create a system of collaborative AI agents that can work together to analyze dessert-related data. Let me walk you through what's happening in this code.

## Preparation

### Install required pacakges

In [ ]:
!pip install -q crewai[tools]==0.86.0 langchain_google_genai PyPDF2 pymupdf
!pip install -q -U duckduckgo-search

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.0/192.0 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 54.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 59.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip freeze | grep "crew\|lang"

crewai==0.86.0
crewai-tools==0.25.8
google-ai-generativelanguage==0.6.17
google-cloud-language==2.17.1
langchain==0.3.24
langchain-cohere==0.3.5
langchain-community==0.3.22
langchain-core==0.3.55
langchain-experimental==0.3.4
langchain-google-genai==2.1.3
langchain-openai==0.2.14
langchain-text-splitters==0.3.8
langcodes==3.5.0
langsmith==0.3.33
language_data==1.3.0
libclang==18.1.1


### Required Packages

In [ ]:
from crewai import Agent, Task, Crew, Process
from PyPDF2 import PdfReader
from crewai.tools import tool
from crewai_tools import FileReadTool, DirectoryReadTool, WebsiteSearchTool, CodeInterpreterTool

import os

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_config.py:323: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
/usr/local/lib/python3.11/dist-packages/pydantic/fields.py:1076: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'example'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warn(
/usr/local/lib/python3.11/dist-packages/crewai_tools/tools/scrapegraph_scrape_tool/scrapegraph_scrape_tool.py:35: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see t

In [ ]:
os.makedirs('./data', exist_ok=True)
os.makedirs('./report', exist_ok=True)

### Data upload
This section fetches dessert-related PDF files and a CSV from a GitHub repository. The data covers various dessert categories such as accessories, cupcakes, desserts, cakes, and cake descriptions in tabular format.

In [ ]:
import requests

links = [
    {"url":'https://raw.githubusercontent.com/IvanReznikov/Generative-AI-Creating-a-LLM-Chatbot-for-Business/refs/heads/main/data/Accessories.pdf', "name": "accessories"},
    {"url":'https://raw.githubusercontent.com/IvanReznikov/Generative-AI-Creating-a-LLM-Chatbot-for-Business/refs/heads/main/data/Cupcakes.pdf', "name": "cupcakes"},
    {"url":'https://raw.githubusercontent.com/IvanReznikov/Generative-AI-Creating-a-LLM-Chatbot-for-Business/refs/heads/main/data/Desserts.pdf', "name": "desserts"},
    {"url":'https://raw.githubusercontent.com/IvanReznikov/Generative-AI-Creating-a-LLM-Chatbot-for-Business/refs/heads/main/data/cakes.pdf', "name": "cakes"},

    {"url":'https://raw.githubusercontent.com/IvanReznikov/Generative-AI-Creating-a-LLM-Chatbot-for-Business/refs/heads/main/data/cake_descriptions.csv', "name": "cake_table"}
]

for link in links:

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36',
    }

    response = requests.get(
        link["url"],
        headers=headers,
    )

    if link['name'] == "cake_table":
      _path = f"./data/{link['name']}.csv"
    else:
      _path = f"./data/{link['name']}.pdf"
    with open(_path, "wb") as f:
      f.write(response.content)

In [ ]:
from google.colab import userdata
import os

os.environ["OPENAI_API_KEY"] = userdata.get("TT_OPENAI_KEY")

In [ ]:
llm = "gpt-4o-mini"

## Tool and Agent Definition

Lets create custom tools for reading PDFs and CSV files that our future agents will need.

In [ ]:
import pandas as pd
from PyPDF2 import PdfReader
from crewai.tools import tool

def read_pdf(pdf_path):
    reader = PdfReader(pdf_path)
    return [page.extract_text() for page in reader.pages]

@tool("PDF reader")
def read_pdf_tool(pdf_path: str) -> list:
    """PDF Reader"""
    return read_pdf(pdf_path)


@tool("CSV reader")
def read_csv_tool(path: str) -> list:
    """CSV reader"""
    return pd.read_csv(path)

# Tools
docs_tool = DirectoryReadTool(directory='./report')
file_tool = FileReadTool()
code_interpreter = CodeInterpreterTool(unsafe_mode=True)

The notebook creates three specialized agents:

- A Python Engineer - capable of coding and data visualization
- An HTML Creator - skilled at creating formatted reports
- A Dessert Analyst - focused on analyzing the dessert data

Each agent has a specific role, goal, backstory, and access to different tools.

In [ ]:
# Agents
engineer = Agent(
    role="Senior Python Developer",
    goal="Craft well-designed and thought-out code",
    backstory="Expert Python developer and visualizer.",
    tools=[code_interpreter],
    allow_delegation=True,
    llm=llm
)

writer = Agent(
    role='HTML Creator',
    goal='Craft HTML reports. Save the report to file once generated',
    backstory='Expert in styling and presenting dessert data beautifully',
    tools=[docs_tool, file_tool],
    verbose=True,
    allow_delegation=False,
    llm=llm
)

scientist = Agent(
    role="Dessert Analyst",
    goal="Understand and categorize cakes and desserts by reading relevant documents",
    backstory="You are a passionate food scientist and dessert researcher.",
    verbose=True,
    allow_delegation=False,
    llm=llm,
    tools=[read_pdf_tool, read_csv_tool]
)

## Task and crew definition
We'll define two main tasks for the agents to perform:
- the first task is for the Dessert Analyst to analyze the PDF files and identify top dessert options in different categories.
- the second task requires the HTML Creator to generate a visual report based on the analyst's findings.

In [ ]:
task1 = Task(
    description=f"""
    Read through the PDF files:
    1. `./data/accessories.pdf`
    2. `./data/desserts.pdf`
    3. `./data/cakes.pdf`
    4. `./data/cupcakes.pdf`

    Extract and summarize:
    - 3 most luxurious cakes
    - 3 most kid-friendly dessert besides cakes
    - 3 best accessory for themed birthday cakes

    Provide your analysis and reasoning.
    """,
    agent=scientist,
    expected_output='Top 3 picks for each subtask with summary and justification.'
)

In [ ]:
task2 = Task(
    description="""
    Based on the dessert analyst’s findings:
    - Read through the ./data/cake_table.csv file
    - Find the number of cakes that contain berries, nuts and berries and nuts.
    - Create an HTML report with the visual summarizing energy and weights for different berries and nuts cakes.
    - Save the HTML to ./report/report.html in landscape layout
    """,
    agent=writer,
    expected_output='Saved HTML report as ./report/report.html'
)

### Create a Crew
Finally, we'll create a "Crew" with the defined agents and tasks. When the crew is kicked off, the agents will work together to complete the assigned tasks in sequence.

In [ ]:
crew = Crew(
  agents=[engineer, scientist, writer],
  tasks=[task1, task2],
  verbose=True
)

In [ ]:
crew

Crew(id=f15e88eb-a87f-495f-bacd-77219f238bcc, process=Process.sequential, number_of_agents=3, number_of_tasks=2)

### Kickoff the crew - let the magic happen

In [ ]:
result = crew.kickoff()

# Agent: Dessert Analyst
## Task: 
    Read through the PDF files:
    1. `./data/accessories.pdf`
    2. `./data/desserts.pdf`
    3. `./data/cakes.pdf`
    4. `./data/cupcakes.pdf`

    Extract and summarize:
    - 3 most luxurious cakes
    - 3 most kid-friendly dessert besides cakes
    - 3 best accessory for themed birthday cakes

    Provide your analysis and reasoning.
    


/usr/local/lib/python3.11/dist-packages/crewai/tools/tool_usage.py:162: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  acceptable_args = tool.args_schema.schema()["properties"].keys()  # type: ignore # Item "None" of "type[BaseModel] | None" has no attribute "schema"




# Agent: Dessert Analyst
## Thought: I need to gather information from the provided PDF files to extract and summarize the required details about luxurious cakes, kid-friendly desserts, and accessories for themed birthday cakes.
## Using tool: PDF reader
## Tool Input: 
"{\"pdf_path\": \"./data/cakes.pdf\"}"
## Tool Output: 
["Cakes descriptions  \n=================================================================  \n1. Torta della Nonna Amore (G)  \nThis heartwarming cake brings the soul of Tuscan kitchens into your home. Torta della \nNonna Amore is inspired by the timeless Italian tradition of Sunday lunches, where love \nis baked into every crumb. Its delicate vanilla sponge is soaked in a subtle limon cello \nsyrup, giving it a tender citrus warmth, while the smooth ricotta -cream filling whispers \nof centuries -old recipes passed down through generations.  \nThe golden pine nuts on top aren’t just a garnish —they’re a tribute to the rolling hills of \nTuscany, where pine trees 

/usr/local/lib/python3.11/dist-packages/crewai/tools/tool_usage.py:162: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  acceptable_args = tool.args_schema.schema()["properties"].keys()  # type: ignore # Item "None" of "type[BaseModel] | None" has no attribute "schema"


 

I encountered an error while trying to use the tool. This was the error: EOF marker not found.
 Tool PDF reader accepts these inputs: Tool Name: PDF reader
Tool Arguments: {'pdf_path': {'description': None, 'type': 'str'}}
Tool Description: PDF Reader



# Agent: Dessert Analyst
## Using tool: PDF reader
## Tool Input: 
"{\"pdf_path\": \"./data/desserts.pdf\"}"
## Tool Output: 

I encountered an error while trying to use the tool. This was the error: EOF marker not found.
 Tool PDF reader accepts these inputs: Tool Name: PDF reader
Tool Arguments: {'pdf_path': {'description': None, 'type': 'str'}}
Tool Description: PDF Reader.
Moving on then. I MUST either use a tool (use one at time) OR give my best final answer not both at the same time. To Use the following format:

Thought: you should always think about what to do
Action: the action to take, should be one of [PDF reader, CSV reader]
Action Input: the input to the action, dictionary enclosed in curly braces
Observation: the resul

...

 

I encountered an error while trying to use the tool. This was the error: EOF marker not found.
 Tool PDF reader accepts these inputs: Tool Name: PDF reader
Tool Arguments: {'pdf_path': {'description': None, 'type': 'str'}}
Tool Description: PDF Reader



# Agent: Dessert Analyst
## Thought: Thought: I need to gather more information from the PDF files, particularly about desserts and accessories for themed cakes, so I will continue extracting the necessary details from the other PDF files I have access to.
## Using tool: PDF reader
## Tool Input: 
"{\"pdf_path\": \"./data/accessories.pdf\"}"
## Tool Output: 

I encountered an error while trying to use the tool. This was the error: EOF marker not found.
 Tool PDF reader accepts these inputs: Tool Name: PDF reader
Tool Arguments: {'pdf_path': {'description': None, 'type': 'str'}}
Tool Description: PDF Reader.
Moving on then. I MUST either use a tool (use one at time) OR give my best final answer not both at the same time. To Use the f

/usr/local/lib/python3.11/dist-packages/crewai/tools/tool_usage.py:162: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  acceptable_args = tool.args_schema.schema()["properties"].keys()  # type: ignore # Item "None" of "type[BaseModel] | None" has no attribute "schema"


 

I encountered an error while trying to use the tool. This was the error: EOF marker not found.
 Tool PDF reader accepts these inputs: Tool Name: PDF reader
Tool Arguments: {'pdf_path': {'description': None, 'type': 'str'}}
Tool Description: PDF Reader



# Agent: Dessert Analyst
## Thought: Thought: Since I've already encountered issues with reading the dessert and accessories PDF files, I should now focus on utilizing the data I have accessed from the cakes PDF file and summarize the requirements from it. I'll categorize the cakes and provide the luxurious options I found there.
## Using tool: PDF reader
## Tool Input: 
"{\"pdf_path\": \"./data/cupcakes.pdf\"}"
## Tool Output: 

I encountered an error while trying to use the tool. This was the error: EOF marker not found.
 Tool PDF reader accepts these inputs: Tool Name: PDF reader
Tool Arguments: {'pdf_path': {'description': None, 'type': 'str'}}
Tool Description: PDF Reader.
Moving on then. I MUST either use a tool (use one at ti

/usr/local/lib/python3.11/dist-packages/crewai/tools/tool_usage.py:162: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  acceptable_args = tool.args_schema.schema()["properties"].keys()  # type: ignore # Item "None" of "type[BaseModel] | None" has no attribute "schema"


 

I encountered an error while trying to use the tool. This was the error: Arguments validation failed: 1 validation error for FileReadToolSchema
file_path
  Field required [type=missing, input_value={'path': './data/cake_table.csv'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing.
 Tool Read a file's content accepts these inputs: A tool that can be used to read None's content.



# Agent: HTML Creator
## Thought: I need to read the contents of the ./data/cake_table.csv file to analyze the number of cakes that contain berries, nuts, and both.
## Using tool: Read a file's content
## Tool Input: 
"{\"path\": \"./data/cake_table.csv\"}"
## Tool Output: 

I encountered an error while trying to use the tool. This was the error: Arguments validation failed: 1 validation error for FileReadToolSchema
file_path
  Field required [type=missing, input_value={'path': './data/cake_table.csv'}, input_type=dict]
    For further information visit https://

In [ ]:
if not os.path.exists("./report/report.html"):
  print("manual save")
  html_content = result.raw

  # Try to split by triple backticks
  parts = html_content.split("```")

  # If triple backticks are found, pick the longest part, otherwise use the whole content
  if len(parts) > 1:
      longest_part = max(parts, key=len).replace("html\n", "")
  else:
      longest_part = html_content

  # Save to file
  with open("./report/report.html", "w", encoding="utf-8") as myfile:
      myfile.write(longest_part)



manual save
